In [ ]:
# CÉLULA 1: Imports e Configurações Iniciais
import os
import hashlib
import logging
from pathlib import Path
from datetime import datetime

from minio import Minio
from minio.error import S3Error
import psycopg2
from psycopg2 import sql
from tqdm.notebook import tqdm  # Usar tqdm.notebook para Jupyter

# --- Configurações ---
# Caminho da montagem do Google Drive dentro do contêiner Jupyter
GDRIVE_MOUNT_PATH = Path("/home/jovyan/work/gdrive_local_mount/")

# Configurações do MinIO
MINIO_ENDPOINT = "minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "senhasegura"
MINIO_BUCKET_RAW = "recepcao-raw"

# Configurações do PostgreSQL
POSTGRES_HOST = "postgres_db"
POSTGRES_PORT = "5432"
POSTGRES_DB = "postgres"
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "senhasegura"

# Configuração do Logging
LOG_DIR = Path("/home/jovyan/work/logs/")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / f"processamento_drive_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)
logger.info("Configurações iniciais e imports carregados.")
logger.info(f"Montagem do Google Drive esperada em: {GDRIVE_MOUNT_PATH}")
logger.info(f"Log será salvo em: {LOG_FILE}")

# Criar tabela de auditoria genérica de downloads (não depende de projetos)
PG_CONN = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)
cursor = PG_CONN.cursor()
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS reception_audit (
        id SERIAL PRIMARY KEY,
        diretorio_origem TEXT NOT NULL,
        caminho_relativo TEXT NOT NULL,
        nome_arquivo TEXT NOT NULL,
        hash_sha256 VARCHAR(64) NOT NULL,
        tamanho_bytes BIGINT NOT NULL,
        data_processamento TIMESTAMPTZ NOT NULL,
        caminho_minio TEXT NOT NULL
    );
    """
)
PG_CONN.commit()
logger.info("Tabela 'reception_audit' verificada/criada com sucesso.")

# CÉLULA 2: Widget de Seleção da Pasta
import ipywidgets as widgets
from IPython.display import display, clear_output

# Inicializa cliente MinIO
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)
if not client.bucket_exists(MINIO_BUCKET_RAW):
    client.make_bucket(MINIO_BUCKET_RAW)

# Lista todas as pastas no mount do Drive
dir_list = []
for root, dirs, _ in os.walk(GDRIVE_MOUNT_PATH):
    for d in dirs:
        rel = os.path.relpath(os.path.join(root, d), GDRIVE_MOUNT_PATH)
        dir_list.append(rel)

dropdown = widgets.Dropdown(options=sorted(dir_list), description="Pasta:")
button = widgets.Button(description="INICIAR DOWNLOAD")
output = widgets.Output()

def selecionar_pasta(btn):
    with output:
        clear_output()
        print(f"Pasta selecionada: {dropdown.value}")
button.on_click(selecionar_pasta)
display(dropdown, button, output)

# CÉLULA 3: Upload ao MinIO + Persistência no reception_audit
from datetime import datetime

def upload_e_auditar(btn):
    with output:
        clear_output()
        selected = dropdown.value
        print(f"Iniciando upload da pasta '{selected}' para bucket '{MINIO_BUCKET_RAW}'...")

        tarefas = []
        base_dir = GDRIVE_MOUNT_PATH / selected
        for root, _, files in os.walk(base_dir):
            for fname in files:
                path_local = os.path.join(root, fname)
                rel_drive = os.path.relpath(path_local, GDRIVE_MOUNT_PATH)
                obj_path = os.path.join(selected, os.path.relpath(path_local, base_dir))
                tarefas.append((path_local, rel_drive, obj_path))

        for local_path, rel_drive, obj_rel in tqdm(tarefas, desc="Upload & Auditoria"):
            object_name = obj_rel.replace(os.sep, "/")
            client.fput_object(MINIO_BUCKET_RAW, object_name, local_path)

            tamanho = os.path.getsize(local_path)
            sha256 = hashlib.sha256()
            with open(local_path, "rb") as f:
                for chunk in iter(lambda: f.read(8192), b""):
                    sha256.update(chunk)
            hash_hex = sha256.hexdigest()

            # Insere no reception_audit
            cursor.execute(
                """
                INSERT INTO reception_audit (
                    diretorio_origem,
                    caminho_relativo,
                    nome_arquivo,
                    hash_sha256,
                    tamanho_bytes,
                    data_processamento,
                    caminho_minio
                ) VALUES (%s, %s, %s, %s, %s, %s, %s);
                """,
                (
                    dropdown.value,
                    rel_drive,
                    os.path.basename(local_path),
                    hash_hex,
                    tamanho,
                    datetime.utcnow(),
                    object_name
                )
            )
            PG_CONN.commit()

        print(f"Processo concluído: {len(tarefas)} arquivos enviados e auditados.")

button.on_click(upload_e_auditar)


2025-05-29 17:04:58,769 - INFO - Configurações iniciais e imports carregados.
2025-05-29 17:04:58,769 - INFO - Montagem do Google Drive esperada em: /home/jovyan/work/gdrive_local_mount
2025-05-29 17:04:58,769 - INFO - Log será salvo em: /home/jovyan/work/logs/processamento_drive_20250529_170458.log
2025-05-29 17:04:58,775 - INFO - Tabela 'reception_audit' verificada/criada com sucesso.


Dropdown(description='Pasta:', options=('0_PROJETOS WRMELO', '0_PROJETOS WRMELO/0. PROJETO MBA', '0_PROJETOS W…

Button(description='INICIAR DOWNLOAD', style=ButtonStyle())

Output()

In [ ]:
# %% [markdown]
# # Notebook A – Recepcao Raw (completo)
# 
# Pipeline: Google Drive local (mount) → Bucket MinIO `reception-raw`
# + Auditoria na tabela `reception_audit`
# + Hash SHA-256 e checagem de existência para evitar duplicidade
# + Widget para seleção e execução manual.

# %%
import os
import hashlib
import pathlib
import logging
import datetime
from pathlib import Path

import pandas as pd
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output

from minio import Minio
from minio.error import S3Error
import psycopg2

# ░░ CONFIGURAÇÕES INICIAIS ░░
GDRIVE_MOUNT_PATH = Path("/home/jovyan/work/gdrive_local_mount/")
MINIO_BUCKET_RAW = "reception-raw"

MINIO = dict(
    endpoint="minio:9000",
    access_key="admin",
    secret_key="senhasegura"
)
POSTGRES = dict(
    host="postgres_db", port=5432,
    dbname="postgres", user="postgres", password="senhasegura"
)

LOG_DIR = Path("/home/jovyan/work/logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)
log_file = LOG_DIR / f"recepcao_{datetime.datetime.now():%Y%m%d_%H%M%S}.log"

logging.basicConfig(level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler(log_file), logging.StreamHandler()])
logger = logging.getLogger(__name__)

logger.info("Notebook A carregado.")

# ░░ CONEXÃO COM BANCO E TABELA ░░
with psycopg2.connect(**POSTGRES) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        CREATE TABLE IF NOT EXISTS reception_audit (
            id SERIAL PRIMARY KEY,
            diretorio_origem TEXT NOT NULL,
            caminho_relativo TEXT NOT NULL,
            nome_arquivo TEXT NOT NULL,
            hash_sha256 VARCHAR(64) NOT NULL,
            tamanho_bytes BIGINT NOT NULL,
            data_processamento TIMESTAMPTZ NOT NULL,
            caminho_minio TEXT NOT NULL,
            UNIQUE (hash_sha256)
        );
        """)
    conn.commit()
logger.info("Tabela 'reception_audit' verificada/criada.")

# ░░ CONEXÃO MINIO E BUCKET ░░
client = Minio(
    MINIO["endpoint"],
    access_key=MINIO["access_key"],
    secret_key=MINIO["secret_key"],
    secure=False
)
if not client.bucket_exists(MINIO_BUCKET_RAW):
    client.make_bucket(MINIO_BUCKET_RAW)
logger.info(f"Bucket '{MINIO_BUCKET_RAW}' pronto para uso.")

# %% [markdown]
# ## Widget: Selecionar pasta montada no Google Drive

# %%
pastas = []
for root, dirs, _ in os.walk(GDRIVE_MOUNT_PATH):
    for d in dirs:
        rel = os.path.relpath(os.path.join(root, d), GDRIVE_MOUNT_PATH)
        pastas.append(rel)

sel_pasta = widgets.Dropdown(options=sorted(pastas), description="Pasta:")
bt_iniciar = widgets.Button(description="Iniciar Recepção", button_style="success")
out_widget = widgets.Output()

display(sel_pasta, bt_iniciar, out_widget)

# %% [markdown]
# ## Execução: Upload ao MinIO + auditoria com hash e checagem

# %%
def sha256_file(path: pathlib.Path, chunk: int = 4096) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        for block in iter(lambda: f.read(chunk), b""):
            h.update(block)
    return h.hexdigest()


def processar_pasta(btn):
    with out_widget:
        clear_output()
        pasta = sel_pasta.value
        if not pasta:
            print("❌ Nenhuma pasta selecionada.")
            return

        base_dir = GDRIVE_MOUNT_PATH / pasta
        if not base_dir.exists():
            print(f"❌ Pasta '{pasta}' não encontrada.")
            return

        print(f"🚀 Upload da pasta '{pasta}' para o bucket '{MINIO_BUCKET_RAW}'…")

        registros = []
        for root, _, files in os.walk(base_dir):
            for fname in files:
                path_local = pathlib.Path(root) / fname
                rel_drive = os.path.relpath(path_local, GDRIVE_MOUNT_PATH)
                obj_path   = os.path.relpath(path_local, base_dir)
                registros.append((path_local, rel_drive, obj_path))

        now_utc = datetime.datetime.utcnow()

        for local_path, rel_drive, obj_rel in tqdm(registros, desc="Upload & Auditoria"):
            object_name = f"{pasta}/{obj_rel}".replace(os.sep, "/")

            # Skip se já existe
            try:
                client.stat_object(MINIO_BUCKET_RAW, object_name)
                print(f"⏭️  Pulando: {object_name}")
                continue
            except S3Error:
                pass

            client.fput_object(MINIO_BUCKET_RAW, object_name, str(local_path))
            tamanho = local_path.stat().st_size
            hash_hex = sha256_file(local_path)

            with psycopg2.connect(**POSTGRES) as conn:
                with conn.cursor() as cur:
                    cur.execute(
                        """
                        INSERT INTO reception_audit (
                            diretorio_origem, caminho_relativo, nome_arquivo,
                            hash_sha256, tamanho_bytes, data_processamento, caminho_minio
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                        ON CONFLICT (hash_sha256) DO NOTHING;
                        """,
                        (
                            pasta, rel_drive, local_path.name,
                            hash_hex, tamanho, now_utc,
                            f"{MINIO_BUCKET_RAW}/{object_name}"
                        )
                    )
                conn.commit()

        print(f"✅ {len(registros)} arquivos auditados e enviados.")

bt_iniciar.on_click(processar_pasta)


2025-05-30 18:24:20,651 - INFO - Notebook A carregado.
2025-05-30 18:24:20,659 - INFO - Tabela 'reception_audit' verificada/criada.
2025-05-30 18:24:20,663 - INFO - Bucket 'reception-raw' pronto para uso.


Dropdown(description='Pasta:', options=('0_PROJETOS WRMELO', '0_PROJETOS WRMELO/0. PROJETO MBA', '0_PROJETOS W…

Button(button_style='success', description='Iniciar Recepção', style=ButtonStyle())

Output()

: 

In [ ]:
# ✅ Notebook A – a-recepcao-raw.ipynb (versão otimizada)
# Função: Upload de arquivos do Google Drive para o MinIO + auditoria em lote no PostgreSQL

# CÉLULA 1: Imports e Configurações Iniciais
import os
import hashlib
import logging
from pathlib import Path
from datetime import datetime

from minio import Minio
from minio.error import S3Error
import psycopg2
from psycopg2 import sql
from tqdm.notebook import tqdm  # Usar tqdm.notebook para Jupyter

# --- Configurações ---
GDRIVE_MOUNT_PATH = Path("/home/jovyan/work/gdrive_local_mount/")
MINIO_ENDPOINT = "minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "senhasegura"
MINIO_BUCKET_RAW = "reception-raw"

POSTGRES_HOST = "postgres_db"
POSTGRES_PORT = "5432"
POSTGRES_DB = "postgres"
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "senhasegura"

LOG_DIR = Path("/home/jovyan/work/logs/")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / f"processamento_drive_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)
logger.info("Configurações iniciais carregadas.")

PG_CONN = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)
cursor = PG_CONN.cursor()
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS reception_audit (
        id SERIAL PRIMARY KEY,
        diretorio_origem TEXT NOT NULL,
        caminho_relativo TEXT NOT NULL,
        nome_arquivo TEXT NOT NULL,
        hash_sha256 VARCHAR(64) NOT NULL,
        tamanho_bytes BIGINT NOT NULL,
        data_processamento TIMESTAMPTZ NOT NULL,
        caminho_minio TEXT NOT NULL
    );
    """
)
PG_CONN.commit()
logger.info("Tabela 'reception_audit' verificada/criada com sucesso.")

# CÉLULA 2: Widget de Seleção da Pasta
import ipywidgets as widgets
from IPython.display import display, clear_output

client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)
if not client.bucket_exists(MINIO_BUCKET_RAW):
    client.make_bucket(MINIO_BUCKET_RAW)

# Lista de pastas
folder_options = []
for root, dirs, _ in os.walk(GDRIVE_MOUNT_PATH):
    for d in dirs:
        rel = os.path.relpath(os.path.join(root, d), GDRIVE_MOUNT_PATH)
        folder_options.append(rel)

folder_dropdown = widgets.Dropdown(options=sorted(folder_options), description="Pasta:")
run_button = widgets.Button(description="INICIAR UPLOAD")
output = widgets.Output()

# CÉLULA 3: Upload e Auditoria em Batch
from datetime import datetime

def upload_e_auditar(btn):
    with output:
        clear_output()
        selected = folder_dropdown.value
        base_dir = GDRIVE_MOUNT_PATH / selected
        print(f"Iniciando upload da pasta '{selected}' para bucket '{MINIO_BUCKET_RAW}'...")

        tarefas = []
        dados_para_inserir = []

        for root, _, files in os.walk(base_dir):
            for fname in files:
                path_local = os.path.join(root, fname)
                rel_drive = os.path.relpath(path_local, GDRIVE_MOUNT_PATH)
                obj_path = os.path.join(selected, os.path.relpath(path_local, base_dir))
                object_name = obj_path.replace(os.sep, "/")

                client.fput_object(MINIO_BUCKET_RAW, object_name, path_local)

                tamanho = os.path.getsize(path_local)
                sha256 = hashlib.sha256()
                with open(path_local, "rb") as f:
                    for chunk in iter(lambda: f.read(8192), b""):
                        sha256.update(chunk)
                hash_hex = sha256.hexdigest()

                dados_para_inserir.append((
                    selected, rel_drive, os.path.basename(path_local),
                    hash_hex, tamanho, datetime.utcnow(), object_name
                ))

        # Inserção em lote
        cursor.executemany(
            """
            INSERT INTO reception_audit (
                diretorio_origem, caminho_relativo, nome_arquivo,
                hash_sha256, tamanho_bytes, data_processamento, caminho_minio
            ) VALUES (%s, %s, %s, %s, %s, %s, %s);
            """,
            dados_para_inserir
        )
        PG_CONN.commit()

        print(f"✅ Upload concluído: {len(dados_para_inserir)} arquivos enviados e auditados.")

run_button.on_click(upload_e_auditar)
display(folder_dropdown, run_button, output)


2025-05-30 18:44:59,075 - INFO - Configurações iniciais carregadas.
2025-05-30 18:44:59,083 - INFO - Tabela 'reception_audit' verificada/criada com sucesso.


Dropdown(description='Pasta:', options=('0_PROJETOS WRMELO', '0_PROJETOS WRMELO/0. PROJETO MBA', '0_PROJETOS W…

Button(description='INICIAR UPLOAD', style=ButtonStyle())

Output()

: 

In [2]:
# ✅ Notebook A – a-recepcao-raw.ipynb (única célula com barra de progresso e botão verde)

import os
import hashlib
import logging
from pathlib import Path
from datetime import datetime

from minio import Minio
from minio.error import S3Error
import psycopg2
from psycopg2 import sql
from tqdm.notebook import tqdm

import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Configurações ---
GDRIVE_MOUNT_PATH = Path("/home/jovyan/work/gdrive_local_mount/")
MINIO_ENDPOINT = "minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "senhasegura"
MINIO_BUCKET_RAW = "reception-raw"

POSTGRES_HOST = "postgres_db"
POSTGRES_PORT = "5432"
POSTGRES_DB = "postgres"
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "senhasegura"

# --- Logging ---
LOG_DIR = Path("/home/jovyan/work/logs/")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / f"processamento_drive_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
logger.info("Configurações iniciais carregadas.")

# --- PostgreSQL ---
PG_CONN = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)
cursor = PG_CONN.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS reception_audit (
        id SERIAL PRIMARY KEY,
        diretorio_origem TEXT NOT NULL,
        caminho_relativo TEXT NOT NULL,
        nome_arquivo TEXT NOT NULL,
        hash_sha256 VARCHAR(64) NOT NULL,
        tamanho_bytes BIGINT NOT NULL,
        data_processamento TIMESTAMPTZ NOT NULL,
        caminho_minio TEXT NOT NULL
    );
""")
PG_CONN.commit()
logger.info("Tabela 'reception_audit' verificada/criada com sucesso.")

# --- MinIO ---
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)
if not client.bucket_exists(MINIO_BUCKET_RAW):
    client.make_bucket(MINIO_BUCKET_RAW)

# --- Widget Dropdown e Botão Verde ---
folder_options = []
for root, dirs, _ in os.walk(GDRIVE_MOUNT_PATH):
    for d in dirs:
        rel = os.path.relpath(os.path.join(root, d), GDRIVE_MOUNT_PATH)
        folder_options.append(rel)

folder_dropdown = widgets.Dropdown(options=sorted(folder_options), description="Pasta:")
run_button = widgets.Button(description="INICIAR UPLOAD", button_style='success')  # botão verde
output = widgets.Output()

# --- Função com tqdm ---
def upload_e_auditar(btn):
    with output:
        clear_output()
        selected = folder_dropdown.value
        base_dir = GDRIVE_MOUNT_PATH / selected
        print(f"Iniciando upload da pasta '{selected}' para bucket '{MINIO_BUCKET_RAW}'...")

        tarefas = []
        dados_para_inserir = []

        # Contagem total para barra de progresso
        total_files = sum(len(files) for _, _, files in os.walk(base_dir))
        progresso = tqdm(total=total_files, desc="📤 Upload e auditoria")

        for root, _, files in os.walk(base_dir):
            for fname in files:
                path_local = os.path.join(root, fname)
                rel_drive = os.path.relpath(path_local, GDRIVE_MOUNT_PATH)
                obj_path = os.path.join(selected, os.path.relpath(path_local, base_dir))
                object_name = obj_path.replace(os.sep, "/")

                client.fput_object(MINIO_BUCKET_RAW, object_name, path_local)

                tamanho = os.path.getsize(path_local)
                sha256 = hashlib.sha256()
                with open(path_local, "rb") as f:
                    for chunk in iter(lambda: f.read(8192), b""):
                        sha256.update(chunk)
                hash_hex = sha256.hexdigest()

                dados_para_inserir.append((
                    selected, rel_drive, os.path.basename(path_local),
                    hash_hex, tamanho, datetime.utcnow(), object_name
                ))
                progresso.update(1)

        progresso.close()

        # Inserção em lote
        cursor.executemany("""
            INSERT INTO reception_audit (
                diretorio_origem, caminho_relativo, nome_arquivo,
                hash_sha256, tamanho_bytes, data_processamento, caminho_minio
            ) VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, dados_para_inserir)
        PG_CONN.commit()

        print(f"✅ Upload concluído: {len(dados_para_inserir)} arquivos enviados e auditados.")

run_button.on_click(upload_e_auditar)
display(folder_dropdown, run_button, output)


2025-05-31 07:58:45,275 - INFO - Configurações iniciais carregadas.
2025-05-31 07:58:45,282 - INFO - Tabela 'reception_audit' verificada/criada com sucesso.


Dropdown(description='Pasta:', options=('0_PROJETOS WRMELO', '0_PROJETOS WRMELO/0. PROJETO MBA', '0_PROJETOS W…

Button(button_style='success', description='INICIAR UPLOAD', style=ButtonStyle())

Output()